# Check-GPU-Connect

In [4]:
print("auto connect")

auto connect


In [5]:
import tensorflow as tf
print(tf.__version__)

import torch
print(torch.__version__)  # Should show the PyTorch version
print(torch.version.cuda)  # Should show the CUDA version
print(torch.cuda.is_available())  # Should return True if the GPU is available

# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available")

2.17.0
2.4.0+cu121
12.1
False
GPU is not available


# Data-get-home

In [6]:
!git clone https://github.com/QuanHoangNgoc/Data-home.git Data-home

Cloning into 'Data-home'...
remote: Enumerating objects: 3268, done.
remote: Counting objects: 100% (3268/3268), done.
remote: Compressing objects: 100% (3266/3266), done.
remote: Total 3268 (delta 0), reused 3268 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3268/3268), 27.56 MiB | 19.68 MiB/s, done.


# Content

In [25]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential

def get_hypo_network(wide_in, num_out, freeze=False):
    # Load pre-trained MobileNetV2 without the top classification layer
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(wide_in, wide_in, 3))
    glp = GlobalAveragePooling2D()
    flat = Flatten()
    # Add custom classification layers
    custom = Sequential([Dense(1024, activation='relu'),
        Dropout(rate=0.3),
        Dense(512, activation='relu'),
        Dropout(rate=0.25)])
    predictor = Dense(num_out, activation='softmax')

    # Freeze the layers of the pre-trained model
    if(freeze):
        for layer in base_model.layers: layer.trainable = False

    # Create the model
    x = base_model.output
    x = glp(x)
    x = flat(x)
    x = custom(x)
    y = predictor(x)
    network = tf.keras.models.Model(inputs=base_model.input, outputs=y)

    # Print the model summary
    network.summary()
    return network

In [26]:
def compile_network(network, lr=1e-4):
    algo = tf.keras.optimizers.Adam(learning_rate=lr)
    network.compile(optimizer=algo, loss='categorical_crossentropy', metrics=['accuracy', 'recall', 'precision'])
    network.summary()
    return network

In [38]:
import datetime, os
from tensorflow.keras.callbacks import ModelCheckpoint


def get_checkpoint():
    now = datetime.datetime.now()
    day = now.day
    hour = now.hour
    minute = now.minute
    second = now.second
    ip_cp = f"{day}.{hour}.{minute}.{second}"
    print(ip_cp, flush=True)
    path = os.path.join(os.getcwd(), 'CP', f"best-model-{ip_cp}.keras")
    return ModelCheckpoint(path,
        monitor='val_accuracy',
        save_best_only=True, mode='max',
        verbose=1)


def fit_network(network, gen_train, ep, gen_val):
    cp = get_checkpoint()
    history = network.fit(gen_train, epochs=ep, validation_data=gen_val, shuffle=False, callbacks=[cp])
    return history

In [28]:
net = get_hypo_network(224, 10)
net = compile_network(net)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 4,099,658 (15.64 MB)

 Trainable params: 4,065,546 (15.51 MB)

 Non-trainable params: 34,112 (133.25 KB)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 4,099,658 (15.64 MB)

 Trainable params: 4,065,546 (15.51 MB)

 Non-trainable params: 34,112 (133.25 KB)

## Provide metadata

In [29]:
FOLDER = "/content/Data-home/MY_data/train"

In [33]:
NAME_LABELS = [x.lower() for x in sorted(set(os.listdir(FOLDER)))]
NAME_LABELS

['apple',
 'banana',
 'avocado',
 'cherry',
 'kiwi',
 'mango',
 'orange',
 'pinenapple',
 'strawberries',
 'watermelon']

In [45]:
import pandas as pd
import re


def get_metadata(root_folder):
    paths = []
    folders = []
    labels = []
    for dirpath, dirnames, filenames in sorted(os.walk(root_folder)): #!!! must sorted
        for file in filenames:
            fullpath = os.path.join(dirpath, file)
            fullpath = fullpath.replace("\\", "/")
            paths.append(fullpath)

            folder = fullpath.split("/")[-2]
            folder = folder.lower()
            folders.append(folder)

            label = NAME_LABELS.index(folder)
            labels.append(str(label))

    # print(paths, folders)
    meta = pd.DataFrame({'path': paths, 'folder': folders, 'y': labels})
    return meta

## Provide DataGenor

In [46]:
def get_genset(metadata, wide_img, bs, val_mode=False):
    genor_train = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255, #!!!
        horizontal_flip=True,
        rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0)

    genor_val = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255)

    if(val_mode):
        genor = genor_val
        shuff = False
    else:
        genor = genor_train
        shuff = True

    return genor.flow_from_dataframe(
        dataframe=metadata,
        x_col='path',
        y_col='y',
        target_size=(wide_img, wide_img),
        batch_size=bs,
        class_mode='categorical',
        shuffle=shuff)

## Run Script

In [47]:
def format_path(path):
  return path.replace("\\", "/")

_meta1 = format_path("/content/Data-home/MY_data/train")
_meta1 = get_metadata(_meta1)
_meta2 = format_path("/content/Data-home/MY_data/test")
_meta2 = get_metadata(_meta2)
meta = result = pd.concat([_meta1, _meta2], ignore_index=True)
meta

,path,folder,y
0,/content/Data-home/MY_data/train/Apple/img_125...,apple,0
1,/content/Data-home/MY_data/train/Apple/img_423...,apple,0
2,/content/Data-home/MY_data/train/Apple/img_183...,apple,0
3,/content/Data-home/MY_data/train/Apple/img_232...,apple,0
4,/content/Data-home/MY_data/train/Apple/img_514...,apple,0
...,...,...,...
3321,/content/Data-home/MY_data/test/watermelon/img...,watermelon,9
3322,/content/Data-home/MY_data/test/watermelon/img...,watermelon,9
3323,/content/Data-home/MY_data/test/watermelon/img...,watermelon,9
3324,/content/Data-home/MY_data/test/watermelon/img...,watermelon,9


In [48]:
from sklearn.model_selection import train_test_split

# Split the dataset into train+val and test sets first
meta_train, _temp = train_test_split(meta, test_size=0.2, random_state=42)
# Now split the remaining dataset into train and val sets
meta_val, meta_test = train_test_split(_temp, test_size=0.25, random_state=42)

In [49]:
gen_train = get_genset(meta_train, 224, 32)
gen_val = get_genset(meta_val, 224, 32, val_mode=True)
gen_test = get_genset(meta_test, 224, 32, val_mode=True)

Found 2660 validated image filenames belonging to 10 classes.
Found 499 validated image filenames belonging to 10 classes.
Found 167 validated image filenames belonging to 10 classes.


In [ ]:
network = get_hypo_network(224, 10)
network = compile_network(network)
history = fit_network(network, gen_train, 10, gen_val)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 4,099,658 (15.64 MB)

 Trainable params: 4,065,546 (15.51 MB)

 Non-trainable params: 34,112 (133.25 KB)

...

 Trainable params: 4,065,546 (15.51 MB)

 Non-trainable params: 34,112 (133.25 KB)

29.3.56.41
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 5/84 ━━━━━━━━━━━━━━━━━━━━ 9:16 7s/step - accuracy: 0.0918 - loss: 2.4331 - precision: 0.0000e+00 - recall: 0.0000e+00